In [ ]:
import numpy as np

In [ ]:
# Global index of local DOF
def gl_index(elem_index,local_dof):
    return elem_index + local_dof

In [ ]:
# Boundary conditions
def apply_bcs(K_gl,F_gl,dof,value):
    phi = 1.0
    bignumber = 10.0**15
    K_gl[dof,dof] += bignumber*phi*phi
    F_gl[dof] += bignumber*float(value)*phi  

In [ ]:
# Global stiffness matrix and mas matrix
def GlobalSystem(E,L,A,n_elem,bc1,bc2):
    
    # Element stiffness matrix and element load vector
    k_elem = E*A/L * np.array([[1.0, -1.0], [-1.0, 1.0]])
    f_elem = np.array([0.0, 0.0])

    # Global stiffness matrix and load vector
    K_gl = np.zeros((n_elem+1, n_elem+1))
    F_gl = np.zeros((n_elem+1))
    # Assembly of elements
    for i_el in range(n_elem):
        for i_loc in range(2):
            i_gl = gl_index(i_el, i_loc)
            F_gl[i_gl] += f_elem[i_loc]
            for j_loc in range(2):
                j_gl = gl_index(i_el, j_loc)
                K_gl[i_gl, j_gl] += k_elem[i_loc, j_loc]

    # Boundary conditions
    apply_bcs(K_gl, F_gl, 0, bc1)
    apply_bcs(K_gl, F_gl, n_elem, bc2)

    # Approximate solution
    uh = np.linalg.solve(K_gl, F_gl)

    return K_gl,F_gl,uh

In [ ]:
# Input parameters
# Material parameters
E = 15000.0
L = 2.0
A = 1.0
# Boundary conditions
# Velocities
bc1 = -5.0
bc2 = 5.0
# Mesh
n_elem = 2

In [ ]:
# Test GetGlobalSystem function
GlobalSystem(E,L,A,n_elem,bc1,bc2)

In [ ]:
# Newmark's method
def newmark(K, M, C, u0, v0, acel0, p, n_steps, dt, gamma, beta):
    #acel0 = np.matmul(np.linalg.inv(M),(p0 - C*v0 - K*u0))

    # Degrees of freedom
    dofs = K.shape[0]
    # Integration constants
    alpha = [
        1.0/(beta*dt**2.0),
        gamma/(beta*dt),
        1.0/(beta*dt),
        1.0/(2.0*beta)-1.0,
        gamma/beta-1.0,
        (gamma/beta-2.0)*dt/2.0,
        (1.0-gamma)*dt,
        gamma*dt
        ]

    # Effective stiffness matrix
    Keff = K + alpha[0]*M + alpha[1]*C
    print(Keff)
    # Keff inverse
    Kinv = np.linalg.inv(Keff)
    print(Kinv)

    # Integration
    # Vector of effective forces at time p
    pef = np.zeros((n_steps+1,dofs))
    # Vector of displacements at time u
    u = np.zeros((n_steps+1,dofs))
    u[0] = u0
    # Vector of accelerations  and velocities at time
    acel = np.zeros((n_steps+1,dofs))
    acel[0] = acel0
    v = np.zeros((n_steps+1,dofs))
    v[0] = v0
    for n in range(n_steps):
        # Vector of effective forces p
        term1 = alpha[0]*u[n] + alpha[2]*v[n] + alpha[3]*acel[n]
        term2 = alpha[1]*u[n] + alpha[4]*v[n] + alpha[5]*acel[n]
        pef[n+1] = p[n+1] + np.matmul(M,term1) + np.matmul(C,term2)
        # Vector of displacements u
        u[n+1] = np.matmul(Kinv,pef[n+1])
        # Vector of accelerations acel and velocities v 
        acel[n+1] = alpha[0]*(u[n+1]-u[n]) - alpha[2]*v[n] - alpha[3]*acel[n]
        v[n+1] = v[n] + alpha[6]*acel[n] + alpha[7]*acel[n+1]
    print(pef)  
    
    return u,v,acel

In [ ]:
# Example 9.2 Paultre book (page 244)

M_t = np.array([[5.0*10.0**5.0]])  # kg
K_t = np.array([[3.*10.0**8.0]])  # N/m
C_t = np.array([[1224744.9]])  # Ns/m
dofs = K_t.shape[0]
u0_t = np.array([0.0])
v0_t = np.array([0.0])
acel0 = np.array([0.0])
n_steps = 10
dt_t = 0.05
gamma_t = 1.0/2.0
beta_t = 1.0/4.0

# Function p(t): in this example p is a half cycle sine function  
p_t = np.zeros((n_steps+1, dofs))
n_points = n_steps + 1
l = np.linspace(0, 0.5, n_points)
p_t = np.array([[5000*np.sin(2*np.pi*x) for x in l]])
p_t = np.round(p_t,8)
p_t = np.transpose(p_t)

# Newmark's integration
newmark(K_t, M_t, C_t, u0_t, v0_t, acel0, p_t, n_steps, dt_t, gamma_t, beta_t)

In [ ]:
# Example 22.2 Paultre book (page 683)
M = 20.0*10.0**3 * np.array([[1.0,0.0],[0.0,1.0]])
K = 18.0*10.0**6 * np.array([[2.0,-1.0],[-1.0,1.0]])
C = np.array([[0.0,0.0],[0.0,0.0]])
u0 = np.array([[0.02,0.02]])
v0 = np.array([[0.0,0.0]])
acel0 = np.array([[-18.0,0.0]])
n_steps = 10
p = np.zeros((n_steps+1,dofs))
dt = 0.01
gamma = 1.0/2.0
beta = 1.0/4.0
newmark(K, M, C, u0, v0, acel0, p, n_steps, dt, gamma, beta)

In [2]:
# Fragments sizes
def frag_size_grady(v,Gc,rho,L,t):
    # Strain rate
    strain_rate = np.zeros(t)
    strain_rate = 2.0*v[[t]]/L
    # Fragment size
    s = np.zeros(t)
    for i in range(t):
        s[[i]] = (24*Gc)/(rho*strain_rate[[i]]**2)**(1/3)
    return s

$$
f(x) = a + b\sin (c(x +d)) 
$$

$$
f(x) = b\sin (cx) 
$$

$$
f(x) = 5000\sin (2\pi x) 
$$